<a href="https://colab.research.google.com/github/IYashCanCode/Detectron2-Mask-Detection/blob/main/Streamlit_Mask_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><b>Streamlit API of Mask Segmentation Using Detecton - 2<br>

<br> Project Group : 4 <br>
<br> By:<br>
Neha Chaudhary <br>
Yash Kumar

<b><h1> Installing and loading necessary modules

In [1]:
!git clone https://github.com/facebookresearch/detectron2.git

Cloning into 'detectron2'...
remote: Enumerating objects: 15303, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 15303 (delta 9), reused 11 (delta 1), pack-reused 15275
Receiving objects: 100% (15303/15303), 6.18 MiB | 22.46 MiB/s, done.
Resolving deltas: 100% (11123/11123), done.


In [2]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-_qi4bthx
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-_qi4bthx
  Resolved https://github.com/facebookresearch/detectron2.git to commit 864913f0e57e87a75c8cc0c7d79ecbd774fc669b
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 16.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for detectron2: filename=detectron2-0.6-cp310-cp310-linux_x86_64.whl size=6119640 sha256=644d2fa12ea71b99f748830745f5

In [3]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00


In [4]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.221s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [5]:
!git clone https://github.com/IYashCanCode/Detectron2-Mask-Detection.git

Cloning into 'Detectron2-Mask-Detection'...
remote: Enumerating objects: 500, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 500 (delta 28), reused 20 (delta 6), pack-reused 439
Receiving objects: 100% (500/500), 108.54 MiB | 30.27 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [6]:
import os
os.mkdir('/content/Detectron2-Mask-Detection/uploads')

In [7]:
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.structures import BoxMode
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.utils.logger import setup_logger
setup_logger()

import os
import cv2
import pickle
import random
from PIL import Image
import streamlit as st
import matplotlib.pyplot as plt
from flask import Flask,request,render_template,jsonify,send_file
from google.colab.patches import cv2_imshow

<h3><b>Registering Training images coco format

In [8]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("OD_Mask", {}, "/content/Detectron2-Mask-Detection/sample_train/train_.json", "/content/Detectron2-Mask-Detection/sample_train/images")

In [9]:
OD_Mask_metadata = MetadataCatalog.get("OD_Mask")
dataset_dicts = DatasetCatalog.get("OD_Mask")

WARNING [01/10 14:25:08 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[01/10 14:25:08 d2.data.datasets.coco]: Loaded 337 images in COCO format from /content/Detectron2-Mask-Detection/sample_train/train_.json


In [10]:
OD_Mask_metadata

namespace(name='OD_Mask',
          json_file='/content/Detectron2-Mask-Detection/sample_train/train_.json',
          image_root='/content/Detectron2-Mask-Detection/sample_train/images',
          evaluator_type='coco',
          thing_classes=['Mask', 'No Mask'],
          thing_dataset_id_to_contiguous_id={0: 0, 1: 1})

<h3><b>Training Detectron2 detection model on custome dataset of mask detection

In [11]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml'))
cfg.DATASETS.TRAIN = ("OD_Mask",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url('COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml')
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.02
cfg.MODEL.DEVICE='cuda'
cfg.SOLVER.MAX_ITER = 500
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[01/10 14:25:10 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_a3ec72.pkl: 254MB [00:02, 106MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


[01/10 14:25:13 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[01/10 14:25:35 d2.utils.events]:  eta: 0:05:27  iter: 19  total_loss: 1.876  loss_cls: 0.6955  loss_box_reg: 0.458  loss_mask: 0.6457  loss_rpn_cls: 0.07389  loss_rpn_loc: 0.01669    time: 0.6749  last_time: 0.7189  data_time: 0.0323  last_data_time: 0.0146   lr: 0.00077924  max_mem: 3654M
[01/10 14:25:51 d2.utils.events]:  eta: 0:05:19  iter: 39  total_loss: 1.213  loss_cls: 0.2888  loss_box_reg: 0.56  loss_mask: 0.3492  loss_rpn_cls: 0.03351  loss_rpn_loc: 0.0114    time: 0.6809  last_time: 0.7227  data_time: 0.0130  last_data_time: 0.0232   lr: 0.0015784  max_mem: 3654M
[01/10 14:26:05 d2.utils.events]:  eta: 0:05:06  iter: 59  total_loss: 1.015  loss_cls: 0.1463  loss_box_reg: 0.5092  loss_mask: 0.3503  loss_rpn_cls: 0.009952  loss_rpn_loc: 0.01325    time: 0.6808  last_time: 0.7040  data_time: 0.0194  last_data_time: 0.0014   lr: 0.0023776  max_mem: 3654M
[01/10 14:26:19 d2.utils.events]:  eta: 0:04:55  iter: 79  total_loss: 1.072  loss_cls: 0.194  loss_box_reg: 0.5062  loss_mask

<h2><b>Streamlit API for custom mask segmentation detectron2 model build<br><br><h5><i><b>Please use the link and enter the number sepearted by four ' . '(dots)  in the new opened tab to continue. <br>Example: 34.0.123.177

In [1]:
%%writefile app2.py

from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.structures import BoxMode
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.utils.logger import setup_logger
setup_logger()

import os
import cv2
import pickle
import random
from PIL import Image
import streamlit as st
import matplotlib.pyplot as plt
from flask import Flask,request,render_template,jsonify,send_file
from google.colab.patches import cv2_imshow






setup_logger()
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml'))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url('COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml')
cfg.MODEL.DEVICE='cuda'
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR,'model_final.pth')
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.75
predictor = DefaultPredictor(cfg)

st.title("Detectron2 Mask Segmentation Web App")


uploaded_image = st.file_uploader("Choose an image...", type="jpg")



if uploaded_image is not None:
    image = Image.open(uploaded_image)
    st.image(image, caption="Uploaded Image.", use_column_width=True)
    image.save('./Detectron2-Mask-Detection/uploads/image1.jpg')

    # Add a "Predict" button
    if st.button("Predict"):
        im = cv2.imread('./Detectron2-Mask-Detection/uploads/image1.jpg')
        outputs = predictor(im)
        v = Visualizer(im[:, :, ::-1],
                      metadata={'thing_classes':['Mask', 'No Mask']},
                      scale=1.,
                      instance_mode=ColorMode.IMAGE
                      )
        v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        v.save('./Detectron2-Mask-Detection/uploads/annotated_images_1.jpg')
        st.image('./Detectron2-Mask-Detection/uploads/annotated_images_1.jpg', use_column_width=True)

Overwriting app2.py


In [2]:
!streamlit run app2.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.73.23.73
npx: installed 22 in 3.524s
your url is: https://early-poets-listen.loca.lt
